## RF. Performance assessment.


## Import Libraries:

In [147]:
import numpy as np
import pandas as pd
import datetime
from pathlib import Path

import pickle

import sys
sys.path.append( '..')


In [148]:
layer = 'RNFL' # 'RNFL' # 'GCL'
eye = 'Random' # 'L', 'R', 'Both', 'Random'


In [149]:
df = pd.read_csv( '../../Bases_Datos_Tabulares_Elena_Nov2022/BaseDatos_PPole_EM_2020.csv' )    

In [150]:
# Count

def count( df, g ):
    
    dfb = df.copy()
    
    df = df[df['GRUPO (0=Sano, 1=EM)'] == g]
    df = df[df['OJO (D=1, I=2)']== 1]

    print( 'R')

    print(df.shape)
    print(df.groupby('APELLIDOS').head(1).shape)

    df = dfb
    
    df = df[df['GRUPO (0=Sano, 1=EM)'] == g]
    df = df[df['OJO (D=1, I=2)']== 2]


    print( 'L')

    print(df.shape)
    print(df.groupby('APELLIDOS').head(1).shape)

In [151]:
count(df, 0)
count(df, 1)

R
(34, 159)
(32, 159)
L
(32, 159)
(31, 159)
R
(50, 159)
(46, 159)
L
(50, 159)
(47, 159)


In [152]:
import re

def RNFL_features(df, features):
    
    regex = re.compile( 'Mean [0-9].[0-9] RNFL' )
    filtered = [f for f in features if regex.match(f)]

    for f in filtered:
        df[f] = pd.to_numeric(df[f], errors='coerce')
        
    print(df.shape)

    df = df.dropna(subset = filtered, how = 'all') 

    df = df.fillna(0)

    print(df.shape)

    return df, filtered


def GCL_features(df, features):
    
    regex = re.compile( 'Mean [0-9].[0-9] GCL' )
    filtered = [f for f in features if regex.match(f)]

    for f in filtered:
        df[f] = pd.to_numeric(df[f], errors='coerce')
        
    print(df.shape)

    df = df.dropna(subset = filtered, how = 'all') 

    df = df.fillna(0)

    print(df.shape)

    return df, filtered

In [153]:
features = df.keys()

if layer == 'RNFL':    
    df, filtered = RNFL_features(df, features)
elif layer == 'GCL':
    df, filtered = GCL_features(df, features)


(166, 159)
(166, 159)


In [154]:
count(df, 0)
count(df, 1)

R
(34, 159)
(32, 159)
L
(32, 159)
(31, 159)
R
(50, 159)
(46, 159)
L
(50, 159)
(47, 159)


In [155]:
from random import randint

import itertools

if eye == 'R':
    df = df[df['OJO (D=1, I=2)'] == 1]
elif eye == 'L':
    df = df[df['OJO (D=1, I=2)'] == 2]
elif eye == 'Both':
    print( 'Nothing' )
elif eye == 'Random':

    patients = df['APELLIDOS'].unique()
    
    indexes = []
    
    for p in patients:
        
        r = randint(1, 2)
            
        record = df[df['APELLIDOS'] == p]
        
        if record.shape[0] > 1:
                    
            record = record[record['OJO (D=1, I=2)'] == r]
            
            if record.shape[0] > 2:
                print(record)
            
        indexes.append(record.index.values)
            
    df = df.loc[list(itertools.chain(*indexes))]

In [156]:
sanos = df 
sanos = sanos[sanos['GRUPO (0=Sano, 1=EM)'] == 0]

em = df 
em = em[em['GRUPO (0=Sano, 1=EM)'] == 1]

In [157]:
from random import randint

import itertools

def select_train_test( df, test_subjects ):

    patients = df['APELLIDOS'].unique()

    indexes = []
    
    for p in patients:
        
        if p not in test_subjects.values:
            
            record = df[df['APELLIDOS'] == p]
        
            indexes.append(record.index.values)
            
    train = df.loc[list(itertools.chain(*indexes))]
    
    indexes = []

    for p in patients:
        
        if p in test_subjects.values:
        
            record = df[df['APELLIDOS'] == p]
        
            indexes.append(record.index.values)
    
    test = df.loc[list(itertools.chain(*indexes))]
    
    return train, test


In [158]:
import imblearn
print(imblearn.__version__)

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

prediction = []

for i in range(0,9): 
    
    test_f = '../TestSubjects/PPole/test_%d.txt' % i
    
    #  reading the file
    test_subjects = pd.read_csv(test_f, header = None)

    em_train, em_test = select_train_test(em, test_subjects)
    sanos_train, sanos_test = select_train_test(sanos, test_subjects)
    
    print(em_train.shape)
    print(sanos_train.shape)

    print(em_test.shape)
    print(sanos_test.shape)
    
        
    filtered = list(filtered) # + ['Lastname']
    
    em_train = em_train[filtered]
    sanos_train = sanos_train[filtered]

    em_test = em_test[filtered]
    sanos_test = sanos_test[filtered]

    em_train['Diagnosis'] = 1
    sanos_train['Diagnosis'] = 0

    em_test['Diagnosis'] = 1
    sanos_test['Diagnosis'] = 0
    
    ddf = em_train.append(sanos_train)

    y_train = ddf['Diagnosis']
    train = ddf.drop('Diagnosis', axis=1)

    ddf = em_test.append(sanos_test)

    y_test = ddf['Diagnosis']
    test = ddf.drop('Diagnosis', axis=1)
    

    # transform the dataset
    counter = Counter(y_train)
    print(counter)

    oversample = SMOTE()
    train, y_train = oversample.fit_resample(train, y_train)

    counter = Counter(y_train)
    print(counter)

    # --

    # Default parameters

    model = RandomForestClassifier()

    model.fit( train, y_train )
    
    classification = model.predict_proba(test)
        
    ntest = test.shape[0]
    hardEstimClass = -1 * np.ones(ntest, int)

    for s in range(ntest):
        hardEstimClass[s] = np.argmax([classification[s][0], classification[s][1]])
        
    prediction.append( [test_f, classification, hardEstimClass, y_test] )
    

0.8.1
(44, 159)
(30, 159)
(15, 159)
(3, 159)
Counter({1: 44, 0: 30})
Counter({1: 44, 0: 44})
(46, 159)
(28, 159)
(13, 159)
(5, 159)
Counter({1: 46, 0: 28})
Counter({1: 46, 0: 46})
(48, 159)
(26, 159)
(11, 159)
(7, 159)
Counter({1: 48, 0: 26})
Counter({1: 48, 0: 48})
(47, 159)
(24, 159)
(12, 159)
(9, 159)
Counter({1: 47, 0: 24})
Counter({1: 47, 0: 47})
(49, 159)
(27, 159)
(10, 159)
(6, 159)
Counter({1: 49, 0: 27})
Counter({1: 49, 0: 49})
(49, 159)
(25, 159)
(10, 159)
(8, 159)
Counter({1: 49, 0: 25})
Counter({1: 49, 0: 49})
(46, 159)
(25, 159)
(13, 159)
(8, 159)
Counter({1: 46, 0: 25})
Counter({1: 46, 0: 46})
(47, 159)
(26, 159)
(12, 159)
(7, 159)
Counter({1: 47, 0: 26})
Counter({1: 47, 0: 47})
(46, 159)
(28, 159)
(13, 159)
(5, 159)
Counter({1: 46, 0: 28})
Counter({1: 46, 0: 46})


In [159]:
from BCA import calcBCA
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report

def evaluation(prediction):

    estimLabels = []
    trueLabels = []

    # https://uberpython.wordpress.com/2012/01/01/precision-recall-sensitivity-and-specificity/
    for i in range(len(prediction[2])):
        estimLabels += [prediction[2][i]]
        trueLabels += [prediction[3].values[i]]

    estimLabels = np.array(estimLabels)    
    trueLabels = np.array(trueLabels)    

    calcBCA(estimLabels, trueLabels, 2)

    # https://stackoverflow.com/questions/58373488/why-does-my-sklearn-metrics-confusion-matrix-output-look-transposed
    conf = confusion_matrix(trueLabels, estimLabels, [0,1] )

    # Estoy con la clase 1
    tp = conf[1][1]
    tn = conf[0][0]
    fp = conf[0][1]
    fn = conf[1][0]

    prec = precision_score(trueLabels, estimLabels, [1])

    recall = recall_score(trueLabels, estimLabels, [1])

    acc = accuracy_score(trueLabels, estimLabels, [1])

    bca = balanced_accuracy_score(trueLabels, estimLabels)

    f1 = f1_score(trueLabels, estimLabels, [1])

    auc = roc_auc_score(trueLabels, estimLabels )

    sensitivity = (1. * tp)/(tp+fn)
    specificity = (1. * tn)/(tn+fp)


    print( "Confusion" )
    print(conf)

    print( "prec, recall, acc, bca")
    print(prec)
    print(recall)
    print(acc)
    print(bca)

    print( "f1, auc")
    print(f1)
    print(auc)
 
    print(sensitivity, specificity)

    print(classification_report(estimLabels, trueLabels, [0,1]))


    """
    sensitivity = recall = tp / t = tp / (tp + fn)
    specificity = tn / n = tn / (tn + fp)
    precision = tp / p = tp / (tp + fp)
    """

    """
    Formato para latex:  Accuracy (\%) & Sensitivity (\%) & Specificity (\%) & F1-score (\%) & AUC 
    """

    print( layer, " & " , "{:.2f}".format(acc*100), " & ", "{:.2f}".format(sensitivity*100), " & ", "{:.2f}".format(specificity*100), " & ", "{:.2f}".format(f1*100), " & ", "{:.2f}".format(auc*100) )
    
    return acc, sensitivity, specificity, f1, auc
    

In [160]:
COLUMN_NAMES = ['acc', 'sens', 'spec', 'f1', 'auc']

eval_df = pd.DataFrame(columns=COLUMN_NAMES)

for i in range(0,9):
    
    acc, sensitivity, specificity, f1, auc = evaluation(prediction[i])

    print(acc)
    
    new_row = {'acc':acc, 'sens':sensitivity, 'spec':specificity, 'f1':f1, 'auc': auc}
    
    print(new_row)
    
    #append row to the dataframe
    eval_df = eval_df.append(new_row, ignore_index=True)


Class 0 bcaCurr 0.566667 TP 1.000000 TN 12.000000 FP 3.000000 FN 2.000000 acc 0.722222 sens 0.333333 spec 0.800000
Class 1 bcaCurr 0.566667 TP 12.000000 TN 1.000000 FP 2.000000 FN 3.000000 acc 0.722222 sens 0.800000 spec 0.333333
Confusion
[[ 1  2]
 [ 3 12]]
prec, recall, acc, bca
0.8571428571428571
0.8
0.7222222222222222
0.5666666666666667
f1, auc
0.8275862068965518
0.5666666666666667
0.8 0.3333333333333333
              precision    recall  f1-score   support

           0       0.33      0.25      0.29         4
           1       0.80      0.86      0.83        14

    accuracy                           0.72        18
   macro avg       0.57      0.55      0.56        18
weighted avg       0.70      0.72      0.71        18

RNFL  &  72.22  &  80.00  &  33.33  &  82.76  &  56.67
0.7222222222222222
{'acc': 0.7222222222222222, 'sens': 0.8, 'spec': 0.3333333333333333, 'f1': 0.8275862068965518, 'auc': 0.5666666666666667}
Class 0 bcaCurr 0.546154 TP 2.000000 TN 9.000000 FP 4.000000 FN 3

/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[0, 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=[1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/home/monica/.virtualenvs/tadpole/lib64/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass normalize=[1] as keywor

In [161]:
m = eval_df.mean()
s = eval_df.std()

print( layer, " & ", "{:.2f}".format(m['acc']*100), "$\pm$", "{:.2f}".format(s['acc']*100), " & ",  end='')
print( "{:.2f}".format(m['sens']*100), "$\pm$", "{:.2f}".format(s['sens']*100), " & ",  end='' )
print( "{:.2f}".format(m['spec']*100), "$\pm$", "{:.2f}".format(s['spec']*100), " & ",  end='' )
print( "{:.2f}".format(m['f1']*100), "$\pm$", "{:.2f}".format(s['f1']*100), " & ",  end='' )
print( "{:.2f}".format(m['auc']*100), "$\pm$", "{:.2f}".format(s['auc']*100), " \\\\",  end='' )


RNFL  &  62.08 $\pm$ 8.08  & 72.85 $\pm$ 18.23  & 39.65 $\pm$ 18.54  & 70.08 $\pm$ 11.87  & 56.25 $\pm$ 5.85  \\

In [162]:
m = eval_df.max()
s = eval_df.min()

In [163]:
print( layer, " & ", "{:.2f}".format(m['acc']*100), "-", "{:.2f}".format(s['acc']*100), " & ",  end='')
print( "{:.2f}".format(m['sens']*100), "-", "{:.2f}".format(s['sens']*100), " & ",  end='' )
print( "{:.2f}".format(m['spec']*100), "-", "{:.2f}".format(s['spec']*100), " & ",  end='' )
print( "{:.2f}".format(m['f1']*100), "-", "{:.2f}".format(s['f1']*100), " & ",  end='' )
print( "{:.2f}".format(m['auc']*100), "-", "{:.2f}".format(s['auc']*100), " \\\\",  end='' )



RNFL  &  72.22 - 52.38  & 90.91 - 33.33  & 77.78 - 12.50  & 82.76 - 44.44  & 66.88 - 50.60  \\

In [164]:
s

acc     0.523810
sens    0.333333
spec    0.125000
f1      0.444444
auc     0.505952
dtype: float64